In [1]:
import helperFunctions as hf

import numpy as np
import pandas as pd
import pybaseball as pb

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import TransformerMixin
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, recall_score, precision_score
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.ensemble import GradientBoostingClassifier



In [3]:
pb.cache.enable()
pd.set_option("display.max_columns", None)

In [5]:
pitchData = pb.statcast(start_dt="2024-03-01", end_dt="2025-10-31")

This is a large query, it may take a moment to complete
Skipping offseason dates
Skipping offseason dates


100%|████████████████████████████████████████████████████████████████████████████████| 477/477 [00:51<00:00,  9.26it/s]


In [6]:
# Look at pitch movement columns
mainDF = pitchData[['stand','game_date','player_name','events','description','p_throws','hit_location','bb_type','balls','strikes','pitch_type', 'release_speed', 'pfx_x', 'pfx_z', 'release_spin_rate','plate_x','plate_z','on_3b','on_2b','on_1b','outs_when_up','inning','inning_topbot','release_pos_y','at_bat_number','pitch_number','pitch_name','bat_score','fld_score','post_bat_score','post_fld_score', 'delta_home_win_exp','delta_run_exp','bat_speed','swing_length','pitcher_days_since_prev_game','n_thruorder_pitcher', 'arm_angle', 'batter','pitcher']]

In [7]:
ver = mainDF[mainDF['player_name'] == 'Verlander, Justin'].copy()

In [9]:
ver["pitch_success"] = np.where(
    (
        ver["events"].isin([
            'field_out', 'strikeout', 'field_error', 'fielders_choice',
            'double_play', 'grounded_into_double_play', 'force_out'
        ])
    ) |
    (
        ver["description"].isin(['called_strike', 'swinging_strike'])
    ) |
    (
        ver["description"].isin(['foul']) & (ver['strikes'] != 2)
    ),
    True,
    False
)

In [10]:
conditions = [
    ver["events"].isin(['double_play', 'grounded_into_double_play']),
    ver["events"].isin(['strikeout']), 
    ver["events"].isin(['field_out']),
    ver["description"].isin(['swinging_strike', 'called_strike']),
    ver["description"].isin(['foul', 'foul_tip']),
    ver["description"].isin(['ball']),
    ver["description"].isin(['hit_by_pitch']),
    ver["events"].isin(['single']),  # bad outcomes
    ver["events"].isin(['double']),  # bad outcomes
    ver["events"].isin(['triple']),  # bad outcomes
    ver["events"].isin(['home_run']),  # bad outcomes
]

values = [1.00, 0.95, 0.90, 0.75, 0.70, 0.50, 0.25, 0.25, 0.20, 0.15, 0]

ver["pitch_outcome_score"] = np.select(conditions, values, default=np.nan)

In [12]:
ver["on_3b"] = ver["on_3b"].fillna(0)
ver["on_2b"] = ver["on_2b"].fillna(0)
ver["on_1b"] = ver["on_1b"].fillna(0)

In [13]:
ver["pitch_success"].value_counts()

False    2213
True     2138
Name: pitch_success, dtype: int64

In [14]:
verClean = ver[['p_throws','stand','balls','strikes','pitch_type', 'release_speed', 'pfx_x', 'pfx_z', 'release_spin_rate','plate_x','plate_z','on_3b','on_2b','on_1b','outs_when_up','inning','release_pos_y','at_bat_number','pitch_number','pitch_name','bat_score','fld_score','pitcher_days_since_prev_game','n_thruorder_pitcher', 'arm_angle', 'batter', 'pitch_success','pitch_outcome_score']]

In [15]:
run_etl(start_dt="2024-03-01", end_dt="2025-10-31")

In [17]:
xcols = ['p_throws','stand','balls','strikes','pitch_type', 'release_speed', 'pfx_x', 'pfx_z', 'release_spin_rate','plate_x','plate_z','on_3b','on_2b','on_1b','outs_when_up','inning','release_pos_y','at_bat_number','pitch_number','bat_score','fld_score','pitcher_days_since_prev_game','n_thruorder_pitcher', 'arm_angle', 'batter']
ycol = "pitch_success"

In [18]:
numeric_cols = ['balls','strikes', 'release_speed', 'pfx_x', 'pfx_z', 'release_spin_rate','plate_x','plate_z','outs_when_up','inning','release_pos_y','at_bat_number','pitch_number','bat_score','fld_score','pitcher_days_since_prev_game','n_thruorder_pitcher', 'arm_angle']

In [19]:
categorical_cols = ['p_throws','stand','pitch_type','on_3b','on_2b','on_1b','batter']

In [31]:
train_model(pitch_success, xcols, ycol, categorical_cols, numeric_cols)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('scaler', StandardScaler(),
                                                  ['balls', 'strikes',
                                                   'release_speed', 'pfx_x',
                                                   'pfx_z', 'release_spin_rate',
                                                   'plate_x', 'plate_z',
                                                   'outs_when_up', 'inning',
                                                   'release_pos_y',
                                                   'at_bat_number',
                                                   'pitch_number', 'bat_score',
                                                   'fld_score',
                                                   'pitcher_days_since_prev_game',
                                                   'n_thruorder_pitcher',
                                                   'arm_angle']),
       

In [32]:
test.copy()
test["predicted"] = clf.predict(test[xcols])
test.head()

<ipython-input-32-add9dd9037eb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["predicted"] = clf.predict(test[xcols])


,p_throws,stand,balls,strikes,pitch_type,release_speed,pfx_x,pfx_z,release_spin_rate,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,release_pos_y,at_bat_number,pitch_number,pitch_name,bat_score,fld_score,pitcher_days_since_prev_game,n_thruorder_pitcher,arm_angle,batter,pitch_success,pitch_outcome_score,predicted
902,R,L,1,1,FF,93.4,-0.64,1.79,2477,-0.96,1.46,592626,0,605137,1,3,54.5,21,3,4-Seam Fastball,3,0,6,2,62.2,664983,False,0.50,False
453,R,L,0,0,CU,76.9,0.69,-0.71,2538,-1.17,3.11,0,0,0,0,6,54.4,49,1,Curveball,1,3,5,3,54.6,650859,False,0.50,False
2480,R,L,2,0,CH,84.4,-1.06,0.99,1702,-0.63,3.09,0,0,621566,1,1,54.48,9,3,Changeup,0,0,5,1,48.7,686948,True,0.75,True
441,R,L,0,0,CU,76.5,0.66,-1.12,2576,0.34,2.16,0,0,0,0,6,54.54,38,1,Curveball,1,0,5,3,51.3,669701,True,0.75,True
67,R,L,2,1,FF,91.3,-0.84,1.76,2357,0.39,2.83,0,0,0,0,6,54.54,54,4,4-Seam Fastball,2,6,6,3,61.7,664913,True,0.70,True


In [33]:
actual = test[ycol]
predicted = test["predicted"]

In [34]:
labels = [False,True]
cm = confusion_matrix(actual,predicted,labels)
cm = pd.DataFrame(cm, index=labels, columns=labels)
cm

c:\users\17083\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass labels=[False, True] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


,False,True
False,382,98
True,119,368


In [35]:
accuracy_score(actual,predicted)

0.7755946225439504

In [36]:
sampleRow = pd.Series(verClean[xcols].head(1).T.squeeze())

In [44]:
verlander_pitches = pitchData.loc[pitchData['player_name'] == 'Verlander, Justin', 'pitch_type']

# get unique pitch types and convert to a list

known_pitch_types = train['pitch_type'].astype(str).unique()
pitch_types = verlander_pitches.dropna().unique().tolist()
pitch_types = [str(p) for p in pitch_types if str(p) in known_pitch_types]

In [45]:
hf.recommend_pitch_vectorized_safe(sampleRow, pitch_types, clf, numeric_cols, categorical_cols)

('ST',
 {'ST': 0.4445328614585363,
  'CU': 0.3101709380161206,
  'FF': 0.3101709380161206,
  'CH': 0.3101709380161206,
  'SL': 0.3101709380161206,
  'SI': 0.3101709380161206})

In [ ]:
### TODO: split the notebook into a few different python files and call them from the notebook
###       for example, create an ETL .py, a model training .py, and a predictor .py.